In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('train.csv')
train_end_idx = len(df) 
df_test = pd.read_csv('test.csv')
df_test['RainToday'] = np.zeros(len(df_test), dtype=np.int8)

# 為了處理方便，把 'train.csv' 和 'test.csv' 合併起來，'test.csv'的 RainToday 欄位用 0 補起來。
# 以 train_end_idx 作為 'train.csv' 和 'test.csv' 分界列，
df = pd.concat([df, df_test], sort=False)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70000 entries, 0 to 34843
Data columns (total 22 columns):
Date             55943 non-null object
Location         55883 non-null object
MinTemp          56082 non-null float64
MaxTemp          56174 non-null float64
Evaporation      56019 non-null float64
Sunshine         55954 non-null float64
WindGustDir      52611 non-null object
WindGustSpeed    56166 non-null float64
WindDir9am       52126 non-null object
WindDir3pm       54552 non-null object
WindSpeed9am     55971 non-null float64
WindSpeed3pm     56072 non-null float64
Humidity9am      55944 non-null float64
Humidity3pm      55944 non-null float64
Pressure9am      56036 non-null float64
Pressure3pm      55896 non-null float64
Cloud9am         56090 non-null float64
Cloud3pm         56064 non-null float64
Temp9am          55970 non-null float64
Temp3pm          56063 non-null float64
RainToday        70000 non-null int64
RISK_MM          56129 non-null float64
dtypes: float64(16

In [5]:
df.head()

,Date,Location,MinTemp,MaxTemp,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM
0,2016-07-19,NaN,19.5,NaN,5.69496,0.500000,ENE,61.0,ENE,ENE,...,86.641375,41.810006,1020.4,1021.9,NaN,7.00000,NaN,19.6,0,NaN
1,2010-07-26,Canberra,2.1,15.7,NaN,5.700000,E,26.0,NaN,SE,...,53.275715,76.598048,1023.7,1020.8,4.000000,1.00000,9.0,14.1,0,0.0
2,2014-04-02,Woomera,20.9,36.1,5.69496,NaN,S,39.0,S,SW,...,15.471574,52.163254,1017.1,1014.9,4.167266,4.27812,24.2,33.9,0,0.0
3,2014-11-19,Tuggeranong,11.5,23.9,5.69496,7.993227,NNE,35.0,W,NaN,...,46.442398,67.664733,1010.2,NaN,4.167266,4.27812,13.8,21.8,1,0.2
4,NaN,Hobart,8.2,20.5,3.20000,NaN,N,69.0,N,WNW,...,51.830889,51.770240,NaN,1001.2,7.000000,7.00000,NaN,17.6,0,0.6


In [6]:
[col for col in df.columns if df[col].dtype == np.object]

['Date', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']

In [7]:
print([col for col in df.columns if df[col].dtype == np.object])
# 將非數值欄位拿掉
df = df.drop(columns = [col for col in df.columns if df[col].dtype == np.object])

# 將 missing value 補 0
df = df.fillna(0)

['Date', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm']


# train

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns = ['RainToday']).values[:train_end_idx, :],
    df['RainToday'].values[:train_end_idx], test_size=0.5)
X_ans = df.drop(columns = ['RainToday']).values[train_end_idx:, :]

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score

#train tree model

model = DecisionTreeClassifier(random_state = 0)
model.fit(X_train,y_train)

#predict
y_pred_decision = model.predict(X_test)
print('Accuracy: %f' % accuracy_score(y_test, y_pred_decision))
print('f1-score: %f' % f1_score(y_test, y_pred_decision))


Accuracy: 0.839402
f1-score: 0.272985


In [10]:
ans_pred = model.predict(X_ans)
df_sap = pd.DataFrame(ans_pred.astype(int), columns = ['RainToday'])
df_sap.to_csv('myAns.csv',  index_label = 'Id')

In [13]:
x = 0
for i in ans_pred:
    if i == 1:
        x += 1
x

4141